add blur

In [8]:
import cv2
import os
import numpy as np

def apply_motion_blur(image, kernel_size=15):
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
    kernel = kernel / kernel_size

    blurred = cv2.filter2D(image, -1, kernel)
    return blurred

def process_images(input_folder, output_folder, kernel_size=15):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):  
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)

            blurred_image = apply_motion_blur(image, kernel_size)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, blurred_image)

            print(f"Processed and saved: {output_path}")

input_folder = 'Kvasir-SEG-Blured/images/val'
output_folder = 'Kvasir-SEG-Blured2x/images/val'

process_images(input_folder, output_folder, kernel_size=15)


Processed and saved: Kvasir-SEG-Blured2x/images/val\cju160wshltz10993i1gmqxbe.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju171py4qiha0835u8sl59ds.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju175facms5f0993a5tjikvt.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju1819curo000988pd5xcqme.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju183od81ff608017ekzif89.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju1871y11d6r0799k6cw4yze.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju18849rrsgr0988p90hkygb.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju300m3s04fg0988uzupuf7z.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju302fqq9spc0878rrygyzzz.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju303j5r062k098835zxfds5.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju30525w04r10835ygp257sb.jpg
Processed and saved: Kvasir-SEG-Blured2x/images/val\cju306x7w05nb0835cunv799x.jpg
Processed and sa

deblur

In [13]:
pip install natsort


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import shutil
import math

source_dir = "Kvasir-SEG-Blured1/images/train"
target_base_dir = "Kvasir-SEG-Blured1/images"

os.makedirs(target_base_dir, exist_ok=True)

image_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

total_images = len(image_files)
images_per_folder = math.ceil(total_images / 15)

for i in range(15):
    target_dir = os.path.join(target_base_dir, f"train{i+1}")
    os.makedirs(target_dir, exist_ok=True)
    
    start_idx = i * images_per_folder
    end_idx = start_idx + images_per_folder
    images_to_move = image_files[start_idx:end_idx]
    
    for img in images_to_move:
        shutil.move(os.path.join(source_dir, img), os.path.join(target_dir, img))

print("Images have been successfully split into 15 folders.")


Images have been successfully split into 15 folders.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] 
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

task    = 'Deblurring'
inp_dir = 'doubleblur/images/train'
out_dir = 'doubleblur_deblurred_finetuned/images/train'

os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "poop_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.jpg')), restored)

print(f"Files saved at {out_dir}")

KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:]
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

def stuff(task,piggy, inp_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)

    files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                    + glob(os.path.join(inp_dir, '*.JPG'))
                    + glob(os.path.join(inp_dir, '*.png'))
                    + glob(os.path.join(inp_dir, '*.PNG')))

    if len(files) == 0:
        raise Exception(f"No files found at {inp_dir}")

    load_file = run_path(os.path.join(task, "MPRNet.py"))
    model = load_file['MPRNet']()
    model.cuda()

    weights = os.path.join(task, "pretrained_models", "model_"+piggy.lower()+".pth")
    load_checkpoint(model, weights)
    model.eval()

    img_multiple_of = 8

    for file_ in files:
        img = Image.open(file_).convert('RGB')
        input_ = TF.to_tensor(img).unsqueeze(0).cuda()

        h,w = input_.shape[2], input_.shape[3]
        H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
        padh = H-h if h%img_multiple_of!=0 else 0
        padw = W-w if w%img_multiple_of!=0 else 0
        input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

        with torch.no_grad():
            restored = model(input_)
        restored = restored[0]
        restored = torch.clamp(restored, 0, 1)

        restored = restored[:,:,:h,:w]

        restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
        restored = img_as_ubyte(restored[0])

        f = os.path.splitext(os.path.split(file_)[-1])[0]
        save_img((os.path.join(out_dir, f+'.jpg')), restored)

    print(f"Files saved at {out_dir}")

In [5]:
#stuff('Denoising','Denoising','noisy/images/val','noisy_ft_denoised/images/val')
stuff('Denoising','Denoisingold','noisy/images/val','noisy_denoised/images/val')


Files saved at noisy_denoised/images/val


In [4]:
stuff('Deblurring','deblurring','blurred30x/images/train','blurred30xunblur/images/train')
stuff('Deblurring','deblurring','blurred30x/images/val','blurred30xunblur/images/val')

Files saved at blurred30xunblur/images/train
Files saved at blurred30xunblur/images/val


In [2]:
from ultralytics import YOLO  
import logging


def train_models(yaml_file, model_name='yolov8n-seg.pt', epochs=30, imgsz=640, seed=1, optimizer="Adam", lr0=0.001, lrf=0.00005, close_mosaic=10):
    model = YOLO(model_name)
    
    try:
        print(f"\nTraining model with configuration from: {yaml_file}")
        
        results = model.train(
            data=yaml_file, 
            epochs=epochs, 
            imgsz=imgsz,   
            seed=seed,    
            optimizer=optimizer,  
            lr0=lr0,       
            lrf=lrf,       
            close_mosaic=close_mosaic  
        )
        
        print(f"Training complete for: {yaml_file}")
        print(f"F1 Score not found for {yaml_file}. Check results format.")
    
    except Exception as e:
        print(f"Error occurred while training with {yaml_file}: {e}")


In [6]:
from ultralytics import YOLO 

def diffmodel(yaml, path = 'runs/segment/train13/weights/best.pt'):
    model = YOLO(path)  
    
    results = model.val(data=yaml, imgsz=640)  


    print("Class indices with average precision:", results.ap_class_index)
    print("Average precision for all classes:", results.box.all_ap)
    print("Mean average precision at IoU=0.50:", results.box.map50)
    print("Mean recall:", results.box.mr)

    precision = results.box.mp  
    recall = results.box.mr    

    f1_score = 2 * (precision * recall) / (precision + recall)

    print(f"Name: {yaml}")
    print(f"F1 Score: {f1_score}")

In [3]:
train_models('blurred30xunblur.yaml')


Training model with configuration from: blurred30xunblur.yaml
New https://pypi.org/project/ultralytics/8.3.13 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8  Python-3.10.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=blurred30xunblur.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=1, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agno

train: Scanning C:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\blurred30xunblur\labels\train.cache... 1921 images, 1325 backgrounds, 0 corrupt: 100%|██████████| 1921/1921 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


C:\Users\Ben\AppData\Roaming\Python\Python310\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
C:\Users\Ben\AppData\Roaming\Python\Python310\site-packages\albumentations\core\composition.py:161: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
val: Scanning C:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\blurred30xunblur\labels\val.cache... 327 images, 232 backgrounds, 0 corrupt: 100%|██████████| 327/327 [00:00<?, ?it/s]


Plotting labels to runs\segment\train26\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train26
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      30/30      2.68G     0.7244      1.115     0.4993     0.9957          0        640: 100%|██████████| 121/121 [00:21<00:00,  5.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.78it/s]

                   all        327         96      0.977      0.905      0.939      0.697      0.977      0.905      0.937      0.655



30 epochs completed in 0.236 hours.
Optimizer stripped from runs\segment\train26\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train26\weights\best.pt, 6.8MB

Validating runs\segment\train26\weights\best.pt...
Ultralytics YOLOv8.2.8  Python-3.10.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  2.80it/s]


                   all        327         96       0.98      0.885      0.939      0.706       0.98      0.885      0.938      0.653
Speed: 0.6ms preprocess, 3.4ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\segment\train26
Training complete for: blurred30xunblur.yaml
F1 Score not found for blurred30xunblur.yaml. Check results format.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:]
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

# task    = 'Deblurring'
task    = 'Denoising'
inp_dir = 'Kvasir-SEG-Noise/images/train'
out_dir = 'Kvasir-SEG-DeNoise/images/train'

os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "model_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.jpg')), restored)

print(f"Files saved at {out_dir}")

C:\Users\joshu\AppData\Local\Temp\ipykernel_2972\3257596221.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights)


Files saved at Kvasir-SEG-DeNoise/images/train


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:]
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

# task    = 'Deblurring'
task    = 'Denoising'
inp_dir = 'Kvasir-SEG-Noise/images/val'
out_dir = 'Kvasir-SEG-DeNoise/images/val'

os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "model_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.jpg')), restored)

print(f"Files saved at {out_dir}")

C:\Users\joshu\AppData\Local\Temp\ipykernel_2972\597409225.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights)


Files saved at Kvasir-SEG-DeNoise/images/val


reduce image res

In [2]:
import cv2
import os

input_folder = 'Kvasir-SEG/images/val'  
output_folder = 'Kvasir-SEG-LowRes/images/val' 

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
        file_path = os.path.join(input_folder, filename)
        
        img = cv2.imread(file_path)
        
        resized_img = cv2.resize(img, (50, 50))
        
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resized_img)
        
        print(f"Resized and saved: {filename}")

print("All images have been resized and saved to the new folder.")


Resized and saved: cju160wshltz10993i1gmqxbe.jpg
Resized and saved: cju171py4qiha0835u8sl59ds.jpg
Resized and saved: cju175facms5f0993a5tjikvt.jpg
Resized and saved: cju1819curo000988pd5xcqme.jpg
Resized and saved: cju183od81ff608017ekzif89.jpg
Resized and saved: cju1871y11d6r0799k6cw4yze.jpg
Resized and saved: cju18849rrsgr0988p90hkygb.jpg
Resized and saved: cju300m3s04fg0988uzupuf7z.jpg
Resized and saved: cju302fqq9spc0878rrygyzzz.jpg
Resized and saved: cju303j5r062k098835zxfds5.jpg
Resized and saved: cju30525w04r10835ygp257sb.jpg
Resized and saved: cju306x7w05nb0835cunv799x.jpg
Resized and saved: cju310f6val1v0855xo8tc3gu.jpg
Resized and saved: cju3128yi0rpu0988o4oo5n8n.jpg
Resized and saved: cju320gyvbch60801v2amdi2g.jpg
Resized and saved: cju323ypb1fbb0988gx5rzudb.jpg
Resized and saved: cju324q101fhe08350wae9cif.jpg
Resized and saved: cju326h4v1gxw08352px40p7r.jpg
Resized and saved: cju3280wv1ir009882jze27tc.jpg
Resized and saved: cju330ofbc2l30801th5g3hw6.jpg
Resized and saved: c

upscale image res

In [33]:
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from PIL import Image
import glob
import os
import numpy as np

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                num_block=23, num_grow_ch=32, scale=4)
upscaler = RealESRGANer(scale=4, model_path='RealESRGAN_x4plus.pth', model=model, tile=0, tile_pad=10, pre_pad=0, half=True)

# input_folder = 'Kvasir-SEG/images/train'
input_folder = 'xray/images/train'
output_folder = 'xray/imageupscaled/train'
os.makedirs(output_folder, exist_ok=True)

image_paths = glob.glob(os.path.join(input_folder, '*'))

for image_path in image_paths:
    img = Image.open(image_path).convert('RGB')
    img = np.array(img)

    with torch.no_grad():
        upscaled_img, _ = upscaler.enhance(img, outscale=4)

    upscaled_img = Image.fromarray(upscaled_img)
    base_name = os.path.basename(image_path)
    upscaled_img.save(os.path.join(output_folder, base_name))


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

create masks for yolov8 input

In [1]:
import os

import cv2
import numpy as np

input_dir = 'Kvasir-SEG/masks'
output_dir = 'Kvasir-SEG/polyMasks'

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 1:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

In [4]:
print(torch.cuda.is_available())

True


In [3]:
import torch
print(torch.__version__)


2.4.0+cu124


In [2]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 2.5/2.5 GB ? eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 5.9/5.9 MB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 4.1/4.1 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.1
    Uninstalling torchvision-0.18.1:
      Successfully uninstalled torchvision-0.18.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.1+cu121
    Uninstalling torchaudio-2.3.1+cu121:
      Successfully uninstalled torchaudio-2.3.1+cu121
Note: you may need to restart the kernel to use updated p

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.2.86 requires torch<2.4.0,>=1.8.0; sys_platform == "win32", but you have torch 2.4.0+cu124 which is incompatible.


In [3]:
import torch


if torch.cuda.is_available():
    device = 'cuda'
    print(f"CUDA is available. Training on GPU: {torch.cuda.get_device_name(0)}")
else:
    device = 'cpu'
    print("CUDA is not available. Training on CPU.")


CUDA is available. Training on GPU: NVIDIA GeForce RTX 3060 Ti


In [1]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124Note: you may need to restart the kernel to use updated packages.



train

In [1]:
import torch

torch.cuda.set_device(0) 

In [2]:
from ultralytics import YOLO
# https://docs.ultralytics.com/usage/cfg/#train

model = YOLO('yolov8n-seg.pt')  
# model = YOLO('yolov8s-p2.yaml').load('yolov8s.pt')  
# model.train(data='config.yaml', epochs=100, imgsz=640, seed=1, lr0=0.001, lrf=0.00005, close_mosaic=25)

# model.train(data='config.yaml', epochs=30, imgsz=640, seed=1, optimizer="Adam", lr0=0.001, lrf=0.00005, close_mosaic= 15)
model.train(data='config.yaml', epochs=10, imgsz=640, seed=1, optimizer="Adam", lr0=0.001, lrf=0.00005, close_mosaic=5)

# model.train(data='config.yaml', epochs=20, imgsz=640, seed=1, lr0=0.0001, lrf=0.000005, close_mosaic=10)
# model.train(data='config.yaml', epochs=20, imgsz=640, seed=1)

WARNING  Known issue with torch>=2.4.0 on Windows with CPU, recommend downgrading to torch<=2.3.1 to resolve https://github.com/ultralytics/ultralytics/issues/15049
New https://pypi.org/project/ultralytics/8.2.102 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.86  Python-3.10.6 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=config.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=1, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=5, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=T

c:\Users\joshu\AppData\Local\Programs\Python\Python310\lib\site-packages\ultralytics\engine\trainer.py:268: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning C:\Users\joshu\Documents\0 uni courses\fourth year\760\segmentation\Kvasir-SEG-Noise\labels\train.cache... 828 images, 0 backgrounds, 0 corrupt: 100%|██████████| 828/828 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


c:\Users\joshu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\joshu\AppData\Local\Programs\Python\Python310\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning C:\Users\joshu\Documents\0 uni courses\fourth year\760\segmentation\Kvasir-SEG-Noise\labels\val.cache... 172 images, 0 backgrounds, 0 corrupt: 100%|██████████| 172/172 [00:00<?, ?it/s]


Plotting labels to runs\segment\train9\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train9
Starting training for 10 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        172       1520      0.523      0.103      0.098     0.0672      0.507      0.102     0.0986     0.0661



10 epochs completed in 0.042 hours.
Optimizer stripped from runs\segment\train9\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train9\weights\best.pt, 6.8MB

Validating runs\segment\train9\weights\best.pt...
Ultralytics YOLOv8.2.86  Python-3.10.6 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.54it/s]


                   all        172       1520      0.538        0.1     0.0989     0.0687       0.53     0.0993     0.0973     0.0659
Speed: 0.6ms preprocess, 2.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\segment\train9


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000014325895F00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [3]:
# pip install --upgrade ultralytics


INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   --------------------------------------- 872.0/872.0 kB 13.0 MB/s eta 0:00:00
   ---------------------------------------- 159.8/159.8 MB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 1.2/1.2 MB 4.5 MB/s eta 0:00:00
   --------------------------------------- 228.5/228.5 MB 15.9 MB/s eta 0:00:00
   ---------------------------------------- 3.5/3.5 MB 2.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0+cu121
    Uninstalling torch-2.4.0+cu121:
      Successfully uninstalled torch-2.4.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.0
    Uninstalling torchvision-0.19.0:
      Successfully uninstalled torchvision-0.19.0
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.58
    Uninstalling ultralytics-8.

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.0+cu121 requires torch==2.4.0+cu121, but you have torch 2.3.1 which is incompatible.


load trained model

In [1]:
from ultralytics import YOLO
model = YOLO('\\Users\\... model location')

KeyboardInterrupt: 

evaluate

In [2]:
import torch
from ultralytics import YOLO

model = YOLO('runs/segment/noise10epochs/weights/best.pt') 

results = model.val(data='config.yaml', imgsz=640)

print("Class indices with average precision:", results.ap_class_index)
print("Average precision for all classes:", results.box.all_ap)
print("Mean average precision at IoU=0.50:", results.box.map50)
print("Mean recall:", results.box.mr)

Ultralytics YOLOv8.2.86  Python-3.10.6 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning C:\Users\joshu\Documents\0 uni courses\fourth year\760\segmentation\Kvasir-SEG-Noise\labels\val.cache... 172 images, 0 backgrounds, 0 corrupt: 100%|██████████| 172/172 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.01it/s]


                   all        172       1520      0.532      0.101      0.099     0.0688      0.531        0.1     0.0973     0.0661
Speed: 1.1ms preprocess, 4.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\segment\val39
Class indices with average precision: [0]
Average precision for all classes: [[    0.09898    0.096554    0.093926    0.088849    0.085084    0.079234    0.066791    0.049185    0.027373   0.0022897]]
Mean average precision at IoU=0.50: 0.09897972471045055
Mean recall: 0.1006578947368421


In [23]:
torch.__version__

'2.4.0+cu124'

predict

results = model.predict(source = "data...", save=True)